# Test Notebook

In [1]:
import pandas as pd

df = pd.read_csv("../data/train_data.csv")
df.shape

(26427, 2)

In [2]:
import pandas as pd

df = pd.read_csv("../data/dev_data.csv")
df.shape

(3303, 2)

In [3]:
import pandas as pd

df = pd.read_csv("../data/test_data.csv")
df.shape

(3304, 2)